In [2]:
import numpy as np

In [4]:
class Island:    
    def __init__(self, x, y, radius, sound_id, is_distractor=False):
        self.x = x  # in meters
        self.y = y  # in meters
        self.r = radius  # in meters
        self.sound_id = sound_id
        self.is_distractor = is_distractor

    def __str__(self):
        return ",".join(["%.4f" % x for x in (self.x, self.y, self.r)] + [str(self.sound_id), "1" if self.is_distractor else "0"])        

In [15]:
class IslandFactory:
    
    def __init__(self, floor_radius, angle_corr, cfg_exp):
        # floor_radius in meters, angle_corr in deg
        self.floor_radius = floor_radius
        self.angle_corr = angle_corr
        self.cfg_exp = cfg_exp

        self.is_fixed = not (cfg_exp['target_angle'] == 'random')
        self.phi_initial = np.random.rand() * 2 * np.pi if not self.is_fixed else np.deg2rad(int(cfg_exp['target_angle']))
        self.rho = self.floor_radius - cfg_exp['target_radius']  # constant, in meters        
        self.last_tgt_x, self.last_tgt_y = None, None
        
    def _correct_angle(self, phi):
        return (2*np.pi - phi) + np.deg2rad(self.angle_corr)
    
    def generate_islands(self, time_from_start):
        def get_fixed_island_position(time_from_start):
            # returns the angle of the island (in polar coordinates)
            # all angles are in radians

            # no change during light periods
            if time_from_start <= self.cfg_exp['timepoints'][0] or time_from_start >= self.cfg_exp['timepoints'][-1]:
                return self.phi_initial

            # max anglular conflict in the middle of the dark
            if time_from_start >= self.cfg_exp['timepoints'][1] and time_from_start <= self.cfg_exp['timepoints'][2]:
                return self.phi_initial + np.deg2rad(self.cfg_exp['phi_max'])

            # raising phase
            if time_from_start < self.cfg_exp['timepoints'][1]:
                p_dur = self.cfg_exp['timepoints'][1] - self.cfg_exp['timepoints'][0]
                t_in_p = time_from_start - self.cfg_exp['timepoints'][0]
                return self.phi_initial + np.deg2rad(self.cfg_exp['phi_max']) * (t_in_p/p_dur)

            # falling phase
            p_dur = self.cfg_exp['timepoints'][3] - self.cfg_exp['timepoints'][2]
            t_in_p = time_from_start - self.cfg_exp['timepoints'][2]
            return self.phi_initial + np.deg2rad(self.cfg_exp['phi_max']) * (1 - (t_in_p/p_dur))        
        
        def get_new_xy(new_islands):
            while True:
                # all in meters
                rho = self.rho if self.is_fixed else np.sqrt(np.random.rand()) * (self.floor_radius - self.cfg_exp['target_radius'])
                phi = np.random.rand() * 2 * np.pi
                x = rho * np.sin(phi)
                y = rho * np.cos(phi)

                too_close = False
                for island in new_islands:
                    if (island.x - x)**2 + (island.y - y)**2 < (2*self.cfg_exp['target_radius'])**2:
                        too_close = True
                        break

                if too_close:
                    continue
                return x, y

        # target island
        if self.is_fixed:
            phi = get_fixed_island_position(time_from_start)
            phi = self._correct_angle(phi) # correct for the camera orientation
            x, y = self.rho * np.sin(phi), self.rho * np.cos(phi)
        else:
            x, y = get_new_xy([])
        islands = [Island(x, y, self.cfg_exp['target_radius'], 2, False)]  # sound 2 is always target
        self.last_tgt_x, self.last_tgt_y = x, y

        # distractors
        sound_ids = [3 + i for i in range(self.cfg_exp['distractor_islands'])]
        if self.cfg_exp['distractor_islands'] > 0:
            for i in range(self.cfg_exp['distractor_islands']):
                x, y = get_new_xy(islands)
                islands.append(Island(x, y, self.cfg_exp['target_radius'], sound_ids.pop(np.random.randint(0, len(sound_ids))), True))

        return islands  # always a list of Islands